In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,84773.52,-1.10,-0.22,-0.11,0.01,-0.01,0.00,-0.00
3,-1.10,10927.53,0.09,-0.05,0.01,-0.00,-0.00,0.00
4,-0.22,0.09,4084.46,-0.01,0.03,0.01,-0.00,-0.00
5,-0.11,-0.05,-0.01,942.55,-0.00,-0.00,0.00,0.00
6,0.01,0.01,0.03,-0.00,177.89,0.00,0.00,0.00
7,-0.01,-0.00,0.01,-0.00,0.00,30.09,-0.00,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,4.36,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.39


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00001,0.00000,-0.00001,0.00000,-0.00001
3,-0.00004,1.00000,0.00001,-0.00001,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00004,0.00004,-0.00000,-0.00002
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00001,-0.00001,0.00001,0.00000
6,0.00000,0.00001,0.00004,-0.00001,1.00000,0.00002,0.00000,0.00003
7,-0.00001,-0.00001,0.00004,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00000,-0.00001,-0.00000,0.00001,0.00000,-0.00001,1.00000,0.00002
9,-0.00001,0.00000,-0.00002,0.00000,0.00003,-0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.603627143484934

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.98678087e+08 1.46854611e+07 1.98186367e+06 1.01517129e+05
 3.39375579e+03 8.64139331e+01 1.35933257e+00 1.53330461e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999800,-0.018723,-0.006881,-0.001584,-0.000299,-0.000051,-0.000007,-6.765403e-07
1,0.018155,0.997146,-0.071885,-0.014025,-0.002623,-0.000444,-0.000064,-5.705403e-06
2,0.008103,0.070943,0.996166,-0.049750,-0.008799,-0.001483,-0.000215,-1.937014e-05
3,0.002221,0.017324,0.048114,0.997439,-0.049294,-0.007978,-0.001153,-1.028252e-04
4,0.000522,0.004046,0.010833,0.048205,0.997039,-0.058217,-0.008115,-7.210242e-04
5,0.000118,0.000910,0.002431,0.010559,0.056983,0.995553,-0.073951,-6.384381e-03
6,0.000025,0.000198,0.000528,0.002289,0.012135,0.072487,0.993273,-8.945213e-02
7,0.000005,0.000035,0.000095,0.000410,0.002172,0.012849,0.088730,9.959704e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00020026693976882282,
 0.0028544740024660786,
 0.0038339849545582227,
 0.0025607052368374905,
 0.0029610276172035155,
 0.004447266179893439,
 0.006727086640494484,
 0.004029606179055345]